## Plot Transit Routes with Low Speeds / High Trips 

Plot the transit route's line geom. Not highway segments. See what shows up.

In [ ]:
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown

from bus_service_utils import (calenviroscreen_lehd_utils, 
                               create_parallel_corridors)
from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils

catalog = intake.open_catalog("./*.yml")

In [ ]:
def prep_data_for_viz():
    gdf = catalog.bus_routes_aggregated_stats.read()
    
    # Fix types, rounding
    drop_cols = ["route_length", "bus_difference_mean", 
                 "bus_multiplier_mean", "bus_difference_spread"]
    
    gdf = gdf.assign(
        route_length_mi = round(gdf.route_length / geography_utils.FEET_PER_MI, 2),
        mean_speed_mph = gdf.mean_speed_mph.round(1),
        pct_trips_competitive = gdf.pct_trips_competitive.round(2)
    ).drop(columns = drop_cols).to_crs(geography_utils.WGS84)
    
    integrify_me = ["num_competitive", "trips_all_day", "trips_peak"]
    gdf[integrify_me] = gdf[integrify_me].astype("Int64")
    
    # Missing values present, drop now
    gdf = gdf[gdf.caltrans_district.notna()].reset_index(drop=True)
    
    return gdf

gdf = prep_data_for_viz()

In [ ]:
def make_map(gdf: gpd.GeoDataFrame, district: str,
             trips_col: str, speed_col: str) -> gpd.GeoDataFrame: 
    
    
    table_title = (f"Routes in top 50% of {trips_col.replace('_', ' ')} and "
                   f"bottom 50% of {speed_col.replace('_', ' ')}"
                  )
    
    display(Markdown(f"#### {table_title}"))
    display(gdf)
    
    m = gdf.drop(columns = "service_date").explore(
        "route_id", categorical=True,
        cmap = cp.CALITP_CATEGORY_BOLD_COLORS,
        tiles = "Carto DB Positron"
    )
    
    display(Markdown(f"### District {district} Routes with Low Speeds / High Trips"))
    display(m)


def identify_high_trip_low_speeds_by_district(df: gpd.GeoDataFrame, 
                                              district: str, trips_col: str, 
                                              speed_col: str) -> gpd.GeoDataFrame:
    subset = df[df.caltrans_district == district].reset_index(drop=True)
    
    print(f"# routes: {len(subset)}")
    
    stats_cols = ["trips_peak", "trips_all_day", 
                  "mean_speed_mph", "pct_trips_competitive"]
    
    subset2 = subset[(subset.mean_speed_mph.notna()) & 
                     (subset[trips_col].notna()) & 
                     (subset.geometry.notna())
                    ].reset_index(drop=True)
    
    if len(subset2) > 0:
        print(f"# routes with full info:{len(subset2)}")
        print(f"# routes with missing info: {len(subset) - len(subset2)}")
        
        subset3 = calenviroscreen_lehd_utils.define_equity_groups(
            subset2, percentile_col = stats_cols , num_groups = 4)
        
        # Plot trips that are top 50%, speeds are bottom 50%
        can_be_plotted = subset3[(subset3[f"{trips_col}_group"] >= 3) & 
                                 (subset3[f"{speed_col}_group"] <= 2)
                                ]
        if len(can_be_plotted) > 0:
            make_map(can_be_plotted, district, trips_col, speed_col)

            return subset3
        else:
            print(f"No observations with combination: top 50% of {trips_col}, bottom 50% of {speed_col}")
    else:
        print("No observations after missing info dropped")

In [ ]:
districts = gdf[gdf.caltrans_district.notna()].caltrans_district.unique().tolist()

for d in sorted(districts):
    display(Markdown(f"## District {d}"))
    
    district = identify_high_trip_low_speeds_by_district(
        gdf, d, 
        trips_col = "trips_all_day", speed_col = "mean_speed_mph")